In [15]:
import sys

!{sys.executable} -m pip install -q langchain-core requests
!{sys.executable} -m pip install ipywidgets

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   --------------------------------- ------ 1.8/2.2 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.9 MB/s  0:00:00

   ---------------------------------------- 0/3 [widgetsnbextension]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ------------- 2/3 [ipywidgets]
   -------------------------- ----------

In [48]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [49]:
load_dotenv()

True

In [4]:
# tool create

@tool
def multiply(a : int , b : int) -> int:
    """Given two numbers a and b return their product"""
    return a * b




In [5]:
print(multiply.invoke({'a':3,'b':5}))

15


In [6]:
multiply.name

'multiply'

In [9]:
print(multiply.description)
multiply.args

Given two numbers a and b return their product


{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [10]:
# tool binding


In [50]:
llm = HuggingFaceEndpoint(
    repo_id= 'mistralai/Mixtral-8x7B-Instruct-v0.1',
    task= 'text-generation'
)

# mistral can't do tool calling

llm_model = ChatHuggingFace(llm = llm)

# 

In [51]:
llm_model_with_tools = llm_model.bind_tools([multiply])
# now the llm model has a tool which can multiply two numbers

# Note:- not every llm model can bind tools with itself 

In [22]:
llm_model_with_tools.invoke('Hi, How are you')

AIMessage(content=" Hello! I'm just a computer program, so I don't have the ability to feel emotions or have a physical state like a human does. I'm here to help answer any questions you might have. Is there something specific you would like to ask?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 11, 'total_tokens': 66}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--28c7bf59-a49d-44be-80ce-f86b1e735080-0', usage_metadata={'input_tokens': 11, 'output_tokens': 55, 'total_tokens': 66})

In [52]:
result = llm_model_with_tools.invoke('you must apply the multiply function and then respond whether you can multiply 3 with 10')

# mistral can't do tool calling therefore it automatically gives ai response instead of json schema
# in video, nitish used openai chat , therefore he got schema

In [53]:
print(result)

content=' Yes, I can confirm that I can apply the multiply function to the numbers 3 and 10. The product of 3 and 10 is 30.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 187, 'total_tokens': 225}, 'model_name': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--e8280e0d-a95b-4465-8606-ef6c62b3009d-0' usage_metadata={'input_tokens': 187, 'output_tokens': 38, 'total_tokens': 225}


In [ ]:
type(result)

langchain_core.messages.ai.AIMessage

### *Can't execute codes from here as the Hugging face model 'Mistral' does support tool calling , and campusx used openai chat for tool calling*

desired result = llm_model_with_tools.invoke('can you multiply 3 with 10')
was

result.tool_calls[0] 's output:

{'name': 'multiply',
    'args':{'a':3,'b':10},
    'id' : some string,
    'type':'tool call'
}

In [ ]:
multiply.invoke(result.tool_calls[0])

## Making a currency convertor agent

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency : str,converted_currency : str) -> float:
    """This function will get the conversion factor from base currency to the desired conversion currency, this will give real time curreny conversion"""
    
    url = f"someUrlForApiKey/{base_currency}/{converted_currency}"
    
    response = requests.get(url)
    
    return response.json()

@tool
def convert(base_currency_value:int,conversion_factor:Annotated[float,InjectedToolArg]) -> float:
    """Given a currency conversion rate , this function calculates the target currency value from a given base currency value"""
    
    # Annotated will tell the LLM to not try to fill this argument, the developer will inject this value after running earlier tools
    
    return base_currency_value*conversion_factor    
    
    

In [55]:
# tool binding
llm1 = ChatOpenAI()

In [57]:
llm_with_tools = llm1.bind_tools([get_conversion_factor,convert])

In [59]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 usd into inr')]

In [ ]:
# don't run this cell as i don't have openai api token
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
import json


for tool_call in ai_message.tool_calls :
    # execute the 1st tool and get the value in conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
    
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # this json will load the content of tool_message1 as dictionary
        
    #append this tool message to messages list
    messages.appen(tool_message1)
    # execute the 2nd tool the conversion rate from tool open
    if(tool_call['name'] == 'convert'):
        # fetch the currrent argument
        tool_call['arg']['conversion_factor'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.appen(tool_message2)

In [ ]:
llm_with_tools.invoke(messages).content